# Análise comparativa de resultados


## 1. Preparação dos dados

### 1.1 Configurações iniciais

In [11]:
import pandas as pd
import seaborn as sns

from pathlib import Path

%pip install -U scikit-learn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import (
    OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder, LabelEncoder
)


Note: you may need to restart the kernel to use updated packages.


### 1.2 Obtendo os dados

In [28]:
data_path = Path('../data/raw/data.csv')
dict_path = Path('../data/external/dicionario.csv')

In [32]:
!pwd

'pwd' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [33]:
!ls

'ls' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [27]:
 Ler conjunto de dados
df = (
    pd.read_csv(data_path)
)
 Ler dicionário de dados
df_dict = pd.read_csv(dict_path)

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\external\\dicionario.csv'

In [ ]:
# Separar variáveis

variaveis_interesse = df_dict.query('resposta == False').variavel.to_list()
variavel_resposta = df_dict.query('resposta == True').variavel.to_list()


df_X = df[variaveis_interesse]
df_y = df[variavel_resposta]

# Separar variáveis de interesse por tipo
nominal_columns = df_dict.query('variavel in @variaveis_interesse and subtipo == "Nominal"').variavel.to_list()
continuos_columns = df_dict.query('variavel in @variaveis_interesse and subtipo == "Discreta"').variavel.to_list()

### 1.3 Tratando dados discrepantes

In [ ]:
# criando pipelines para tratar os dados

nominal_preprocessor = Pipeline(steps=[
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes ()
    ('encoding', OneHotEncoder(sparse=False, handle_unknown='ignore')), # Codificação de variáveis
    # Seleção de variáveis
     ('normalization', StandardScaler()) # Normalização de variáveis
])
continuous_preprocessor = Pipeline(steps=[
    # Tratamento de dados discrepantes
    ('missing', KNNImputer(n_neighbors=5)), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', RobustScaler()) # Normalização
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuos', continuous_preprocessor, continuos_columns)
])

preprocessor

In [ ]:
preprocessor.fit(df_X)

X_transformed = preprocessor.transform(df_X)

X_transformed